# Setup

In [ ]:
!pip install -qU bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.9 MB/s eta 0:00:00


In [ ]:
# Standard library imports
import os
import warnings

# Core ML framework
import torch

# Torch backend configurations
# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

# Authentication and API clients
from huggingface_hub import login
from google.colab import userdata

# Transformer-specific imports
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer
)

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

warnings.filterwarnings("ignore")


PyTorch - główny framework do uczenia maszynowego.
> torch.backends.cuda.enable_mem_efficient_sdp(False)
   * Wyłącza memory-efficient attention, czyli optymalizację redukującą zużycie pamięci VRAM podczas obliczeń związanych z SDP.
Ta optymalizacja jest korzystna dla modeli z dużymi sekwencjami, ale może powodować niestabilności numeryczne lub problemy z kompatybilnością.
torch.backends.cuda.enable_flash_sdp(False)

   * Wyłącza flash attention, czyli wysoko zoptymalizowaną implementację SDP dostępną na kartach NVIDIA Ampere (A100) i nowszych.
Flash Attention jest zazwyczaj najszybszą metodą, ale jego wyłączenie może być konieczne, jeśli występują błędy lub gdy korzystasz z niestandardowego układu pamięci tensorów.

> torch.backends.cuda.enable_flash_sdp(False)

* Wyłącza flash attention, czyli wysoko zoptymalizowaną implementację SDP dostępną na kartach NVIDIA Ampere (A100) i nowszych.
* Flash Attention jest zazwyczaj najszybszą metodą, ale jego wyłączenie może być konieczne, jeśli występują błędy lub gdy korzystasz z niestandardowego układu pamięci tensorów.

Z biblioteki transformers importowane są kluczowe klasy:
- AutoModelForCausalLM do wczytywania modeli języka
- AutoTokenizer do przetwarzania tekstu na tokeny
- BitsAndBytesConfig do konfiguracji kwantyzacji modelu
- TextStreamer do strumieniowego generowania tekstu

In [ ]:
login(token = userdata.get('HF_TOKEN') )

In [ ]:
class CFG:
    device = 'cuda'
    model = "speakleash/Bielik-11B-v2.2-Instruct"
    max_tokens = 5000
    temperature = 0.1
    top_k = 5
    top_p = 0.9
    dtype = torch.bfloat16

Model_name wskazuje na Bielik-11B-v2.2-Instruct ze platformy Speakleash. Jest to polski model językowego o 11 miliardach parametrów, dostosowana do pracy z instrukcjami. Cyfra 2.2 w nazwie sugeruje, że jest to druga główna wersja modelu z dwoma pomniejszymi aktualizacjami.

# Funkcje

In [ ]:
def generate(prompt, system = None):

    messages = []

    if system:
        messages.append({"role": "system", "content": system})

    messages.append({"role": "user", "content": prompt})


    tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)
    model_input_ids = tokenizer_output.input_ids.to(CFG.device)
    model_attention_mask = tokenizer_output.attention_mask.to(CFG.device)

    outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens = CFG.max_tokens,
                           do_sample=True if CFG.temperature else False,
                           temperature = CFG.temperature,
                           top_k = CFG.top_k,
                           top_p = CFG.top_p)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens = False)

    return answer

Tokenizer przetwarza przygotowane komunikaty na format zrozumiały dla modelu. Używana jest metoda `apply_chat_template`, która konwertuje komunikaty na sekwencję tokenów. Wynik jest zwracany jako tensor PyTorch (return_tensors="pt") w formie słownika (return_dict=True).

Przetworzone dane są przenoszone na odpowiednie urządzenie obliczeniowe (CPU lub GPU) zdefiniowane w konfiguracji CFG.device. Dotyczy to zarówno identyfikatorów tokenów (input_ids) jak i maski uwagi (attention_mask).

Następuje właściwa generacja tekstu przy użyciu metody model.generate(). Funkcja konfiguruje kilka ważnych parametrów generacji:
- streamer umożliwia strumieniowe generowanie tekstu
- max_new_tokens określa maksymalną długość generowanej odpowiedzi
- do_sample włącza losowość w generacji, ale tylko jeśli ustawiono temperaturę
- temperature kontroluje kreatywność modelu
- top_k i top_p to parametry ograniczające wybór następnego tokenu do najbardziej prawdopodobnych opcji

Na końcu wygenerowane tokeny są dekodowane z powrotem na tekst przy użyciu tokenizera. Parametr `skip_special_tokens=False` oznacza, że specjalne tokeny (jak znaczniki początku i końca sekwencji) nie są usuwane z wyniku.


In [ ]:
def resolve_mistakes(text: str, opcje = ['wszystko'], one_shot = True):

    system = "Jesteś ekspertem zajmującym się poprawą tesktów w języku polskim."

    prompt_poczatek = "Sprawdź poniższy tekst w języku polskim i popraw go. Zwróć szczególną uwagę na:\n\n"

    # Mozliwe opcje do wpisania w drugim argumencie: "literowka", "ortografia", "interpunkcja", "fleksja", "skladnia", "leksykalne", "stylistyka"
    prompt_literowka = "* Literówki (pominięcie, powtórzenie lub wstawienie niewłaściwego znaku)\n\n"
    prompt_ortograficzny = "* Błędy ortograficzne (błędny zapis liter lub ich połączeń, błędna pisownia małą lub wielką literą, błędne stosowanie kropki w skrótach, błędy w pisowni łącznej)\n\n"
    prompt_interpunkcyjny = "* Błędy interpunkcyjne (zbędne użycie znaku interpunkcyjnego, brak znaku interpunkcyjnego, błędne zapisy w przypadku zbiegu znaków interpunkcyjnych, użycie niewłaściwego znaku interpunkcyjnego)\n\n"
    prompt_fleksyjny = "* Błędy fleksyjne (związane z wyborem niewłaściwej formy wyrazu, błędną odmianą lub z błędnym brakiem odmiany)\n\n"
    prompt_skladniowe = "* Błędy składniowe (używanie błędnych form lub konstrukcji w miejscach, których uzupełnienia wymagają wyrazy nadrzędne, błędy w szyku, błędne użycie imiesłowowych równoważników zdań)\n\n"
    prompt_leksykalne = "* Błędy leksykalne (błędne użycie słowa w danej konstrukcji, mylenie wyrazów podobnych brzmieniowo, używanie zbędnych zapożyczeń lub nadużywanie wyrazów obcych, błędy słowotwórcze i powstałe w wyniku skojarzeń słowotwórczych, zniekształcenia związków frazeologicznych)\n\n"
    prompt_stylistyczne = "* Błędy stylistyczne (niedostosowanie form językowych do charakteru i funkcji korespondencji, niezamierzone uczynienie tekstu dwuznacznym)\n\n"

    prompt_one_shot = "Poniżej masz przykład: \n\nTeskt do popawy: 'janusz siedział skólnoy za stołm i milczał .'\n\nPrawdiłowy poprawiony teskt: 'Janusz siedział skulony za stołem i milczał.'"
    promt_koniec = "Pamietaj, aby ingerencja w treść była związana tylko z poprawą błędu i nie ingerowala w sens pierwotnej treści. Zwróć tylko poprawiony tekst bez dodatkowych komentarzy. Poniżej tekst do poprawy:\n\n"

    if opcje[0] == 'wszystko':
        opcje = ['literowka', 'ortografia', 'interpunkcja', 'fleksja', 'skladnia', 'leksykalne', 'stylistyka']

    prompt_calosc = prompt_poczatek + (prompt_literowka if 'literowka' in opcje else '') + (prompt_ortograficzny if 'ortografia' in opcje else '') + (prompt_interpunkcyjny if 'interpunkcja' in opcje else '') + (prompt_fleksyjny if 'fleksja' in opcje else '') + (prompt_skladniowe if 'skladnia' in opcje else '') + (prompt_leksykalne if 'leksykalne' in opcje else '') + (prompt_stylistyczne if 'stylistyka' in opcje else '') + (prompt_one_shot if one_shot else '') + promt_koniec + text

    messages = []
    messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": str(prompt_calosc)})

    tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)
    model_input_ids = tokenizer_output.input_ids.to(CFG.device)
    model_attention_mask = tokenizer_output.attention_mask.to(CFG.device)

    outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=5000,
                           do_sample=False,
                           temperature = CFG.temperature, top_k = CFG.top_k, top_p = CFG.top_p)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens = False)

    return answer

In [ ]:
def change_style(text: str, styl: str):
    system = "Jesteś ekspertem zajmującym się zmianą stylu tekstu w języku polskim."

    prompt_poczatek = "Zmień styl napisanego tekstu na "

    # Mozliwe opcje do wpisania w drugim argumencie: "potoczny", "urzedowy", "publicystyczny", "naukowy", "artystyczny"
    prompt_potoczny = "potoczny."
    prompt_urzedowy = "urzędowy."
    prompt_publicystyczny = "publicystyczny."
    prompt_naukowy = "naukowy."
    prompt_artystyczny = "artystyczny."

    promt_koniec = "\n\nPamietaj, aby ingerencja w treść była związana tylko ze zmianą i nie ingerowala w sens\
                        pierwotnej treści. Zwróć tylko poprawiony tekst bez dodatkowych komentarzy.\
                        Poniżej tekst do do zmiany stylu:\n\n"

    prompt_calosc = prompt_poczatek + (prompt_potoczny if styl == 'potoczny' else '') + (prompt_urzedowy if styl == 'urzedowy' else '')
    prompt_calosc += (prompt_publicystyczny if styl == 'publicystyczny' else '') + (prompt_naukowy if styl == 'naukowy' else '')
    prompt_calosc += (prompt_artystyczny if styl == 'artystyczny' else '') + promt_koniec + text

    messages = []
    messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt_calosc})

    tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)
    model_input_ids = tokenizer_output.input_ids.to(CFG.device)
    model_attention_mask = tokenizer_output.attention_mask.to(CFG.device)

    outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=5000,
                           do_sample=True ,
                           temperature = CFG.temperature,
                           top_k = CFG.top_k,
                           top_p = CFG.top_p)

    answer = tokenizer.batch_decode(outputs, skip_special_tokens=False)

    return answer


# Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

Pierwsza linia tworzy tokenizer dopasowany do naszego modelu Bielik. AutoTokenizer.from_pretrained automatycznie pobiera i konfiguruje odpowiedni tokenizer pasujący do wybranego modelu. Jest to bardzo wygodne, ponieważ każdy model może używać nieco innego sposobu tokenizacji, a ta metoda zapewnia pełną kompatybilność.

Druga linia ustawia token wypełniający (pad_token) na taki sam jak token końca sekwencji (eos_token). Jest to ważne ustawienie, ponieważ podczas przetwarzania wielu sekwencji tekstu jednocześnie, muszą one mieć taką samą długość. Krótsze sekwencje są uzupełniane właśnie tym tokenem wypełniającym. Użycie eos_token jako pad_token jest częstą praktyką, ponieważ model jest już przeszkolony do ignorowania tego tokenu na końcu sekwencji.

Trzecia linia tworzy TextStreamer, który będzie odpowiedzialny za wyświetlanie generowanego tekstu. Parametr skip_prompt=True oznacza, że streamer nie będzie pokazywał tekstu wejściowego (promptu), a tylko generowaną odpowiedź. Skip_special_tokens=True sprawia, że specjalne tokeny kontrolne (jak znaczniki początku czy końca sekwencji) nie będą wyświetlane w końcowym tekście. Dzięki temu użytkownik widzi tylko czysty, czytelny tekst odpowiedzi.

**TextStreamer jest szczególnie użyteczny w interaktywnych zastosowaniach, ponieważ pozwala obserwować proces generowania tekstu w czasie rzeczywistym, zamiast czekać na całą odpowiedź.** Jest to nie tylko bardziej przyjazne dla użytkownika, ale też pozwala szybciej zorientować się, czy model zmierza w odpowiednim kierunku.

In [ ]:
quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype = CFG.dtype
)

## Kwantyzacja modelu – BitsAndBytesConfig

### Opis konfiguracji
Konfiguracja dotyczy kwantyzacji modelu, czyli techniki optymalizacji, która zmniejsza zużycie pamięci przy minimalnym wpływie na jakość działania.

### Jak działa kwantyzacja?
- Modele językowe zazwyczaj używają **float32** (32-bitowych liczb zmiennoprzecinkowych).
- Przykładowo, model o **11 miliardach parametrów** wymaga ogromnych zasobów pamięci.
- **Kwantyzacja 4-bitowa** (`load_in_4bit=True`) redukuje pamięć 8-krotnie, przechowując liczby na 4 bitach zamiast 32.
- **Format obliczeń** (`bnb_4bit_compute_dtype = CFG.dtype`) to `bfloat16`, co pozwala na stabilniejsze obliczenia mimo kwantyzacji.
- Wykorzystuje bibliotekę **bitsandbytes**, specjalizującą się w kwantyzacji modeli.

### Korzyści kwantyzacji
- **Zmniejszone wymagania pamięciowe**:
  - Bez optymalizacji: **~22GB GPU** (11B parametrów × 2 bajty).
  - Po kwantyzacji 4-bitowej: **~5.5GB GPU**.
- **Możliwość uruchamiania dużych modeli na słabszych komputerach**.

Dzięki temu użytkownicy mogą korzystać z modeli wielkości **Bielika (11B)** nawet na sprzęcie z ograniczoną pamięcią GPU.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(CFG.model,
                                            torch_dtype = CFG.dtype,
                                            quantization_config=quantization_config,
                                            low_cpu_mem_usage = True
                                            )

model.generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
email = """
Dzień dobry,
Po wtorkowej wizycie otrzymałem faktórę, jednak potrzebuje małej korekty.
potrzebuje aby w danych nabywcy był dodatkowo adres zamieszkania.

Mój adres zamiszkania (jeżeli kiedyś podawałem , to prawdopodobnie jest nieaktualny bo go niedawno zmieniłem,\
    więc proźba o użycie tego):

ul. Marszałkowska 11C/24
00-001 Warszawa

W załączniku skan aktualnej faktura.

Pozdrawiam
Adam Kowalski
"""

In [ ]:
email_poprawiony = resolve_mistakes(email, opcje = ['wszystko'], one_shot = True)



Poprawiony tekst:

Dzień dobry,
Po wtorkowej wizycie otrzymałem fakturę, jednak potrzebuję małej korekty.
Potrzebuję, aby w danych nabywcy był dodatkowo adres zamieszkania.

Mój adres zamieszkania (jeżeli kiedyś podawałem, to prawdopodobnie jest nieaktualny, bo go niedawno zmieniłem, więc proszę o użycie tego):

ul. Marszałkowska 11C/24
00-001 Warszawa

W załączniku skan aktualnej faktury.

Pozdrawiam
Adam Kowalski


In [ ]:
email_nowy_styl = change_style(email_poprawiony[0], styl = 'urzedowy')



---

Poprawiony tekst w stylu urzędowym:

Szanowni Państwo,

W odniesieniu do faktury nr [numer faktury] z dnia [data], uprzejmie informuję, że wymagana jest korekta danych nabywcy. Proszę o uzupełnienie adresu zamieszkania w sekcji danych nabywcy.

Aktualny adres zamieszkania:
ul. Marszałkowska 11C/24
00-001 Warszawa

W załączeniu przesyłam skan aktualnej faktury.

Z poważaniem,
Adam Kowalski
[Stanowisko/Funkcja]
[Nazwa Firmy]
